In [125]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os
import glob

In [126]:
all_files = glob.glob(os.path.join('csv/', "*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files))
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace('-', '_')
df.drop('unnamed: 64', axis=1, inplace=True)

# Turn dates into datetime.
date_cols = df.columns[df.columns.str.contains('date')]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

def original_sentence_format(x):
    try:
        year = int(x[:3])
        month = int(x[5:7]) / 12
        day = int(x[10:12]) / 365
        return year + month + day
    except:
        return np.nan
df['original_sentence'] = df.original_sentence.apply(original_sentence_format)

/var/folders/7b/prbmv40d5wg4ymrn9dr3c76c0000gn/T/ipykernel_4771/1850306438.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files))
/var/folders/7b/prbmv40d5wg4ymrn9dr3c76c0000gn/T/ipykernel_4771/1850306438.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files))
/var/folders/7b/prbmv40d5wg4ymrn9dr3c76c0000gn/T/ipykernel_4771/1850306438.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files))
/var/folders/7b/prbmv40d5wg4ymrn9dr3c76c0000gn/T/ipykernel_4771/1850306438.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files))


In [132]:
df.columns

Index(['case_cause_nbr', 'full_name', 'alias', 'sex', 'race', 'birthdate',
       'sid', 'judicial_nbr', 'addr_house_nbr', 'house_suf',
       'addr_pre_direction', 'addr_street', 'addr_street_suffix',
       'addr_post_direction', 'addr_unit', 'addr_city', 'addr_state',
       'addr_zip_code', 'addr_zip_plus_4', 'offense_date', 'offense_code',
       'offense_desc', 'offense_type', 'reduced_offense_code',
       'reduced_offense_desc', 'reduced_offense_type', 'location',
       'custody_date', 'complaint_date', 'filing_agency_description',
       'case_date', 'case_desc', 'setting_date', 'setting_type', 'g_jury_date',
       'g_jury_status', 'disposition_date', 'disposition_code',
       'disposition_desc', 'judgement_date', 'judgement_code',
       'judgement_desc', 'sentence_desc', 'sentence', 'sentence_start_date',
       'sentence_end_date', 'fine_amount', 'court_costs', 'court_type',
       'court', 'post_judicial_field', 'post_judicial_date', 'bond_date',
       'bond_status', '

1st goal predicting the time of the sentence (regression) - use subset of those who were sentenced.

2nd categorical variable if they were sentenced or not. 

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)

original_sentence >= 1 

In [131]:
df.offense_type.factorize()

(array([0, 0, 0, ..., 4, 0, 0]),
 Index(['F3', 'F1', 'F2', 'F ', 'FS', 'MA', 'F*', 'MC', 'FC', 'MB', 'F-', '  ',
        'M ', 'F0', 'FA', 'FE', 'FQ', 'U ', 'M-', 'FI', ' 1'],
       dtype='object'))